# Creación y limpieza del dataframe

In [3]:
import pandas as pd
import numpy as np
from time import sleep
import json
from tqdm import tqdm
import os
import sys
import warnings
warnings.filterwarnings('ignore')

# Agregar el directorio 'src' al path
sys.path.append(os.path.abspath('../src'))

# Importar la función suma del módulo auxiliares
import funciones_auxiliares as fa

In [15]:
os.chdir("../datos")
datos = os.listdir()[:18]
df=pd.DataFrame()

for i in datos:
    df_aux = pd.read_csv(f"../datos/{i}")
    # print(i)
    df = pd.concat([df, df_aux])

In [5]:
df.drop(columns=["Unnamed: 0"], inplace=True)
df = df.reindex(columns=['supermercado', 'producto','nombre_producto', 'Día', 'Precio (€)', 'Variación'])

Cambiamos la columna de variación para hacer dos columnas, una indicando si ha habido un aumento o una disminucion y otra con el porcentaje.

In [6]:
df["Variación"] = df["Variación"].apply(lambda x: np.nan if x=='=' else x)

In [7]:
# Pasamos de una columna a dos
df_aux = df["Variación"].str.split(" ", expand=True)
df = pd.concat([df,df_aux], axis=1)

In [8]:
# Eliminamos la columna y renombramos
df.drop(columns=["Variación"], inplace=True)
df.rename(columns={0: 'incremento', 1:"porcentaje"}, inplace=True)

In [9]:
len(df["nombre_producto"].unique())

1597

En incremento dejamos solo si es aumento o decremento y en porcentaje solo el número:

In [10]:
df["incremento"]=df["incremento"].apply(lambda x: x[0] if type(x)==str else x)
df["incremento"]=df["incremento"].map({'+':'aumento', '-':'decremento'})
df["porcentaje"]=df["porcentaje"].str.replace(r"[()%]", "", regex=True)

Ahora sacamos una subcategoría a partir del nombre:

In [11]:
df["nombre_producto"]=df["nombre_producto"].str.lower()
df["producto"]=df["producto"].str.lower()


In [12]:
# Aplicar la función al DataFrame para crear la columna subcategoria
df['subcategoria'] = df.apply(fa.obtener_subcategoria, axis=1)

In [14]:
df=df.reindex(columns=['supermercado', 'producto', 'nombre_producto', 'subcategoria', 'Día', 'Precio (€)',
       'incremento', 'porcentaje'])

In [ ]:
df.to_csv("../datos/df_supermercados.csv")